# Requisitos técnicos

In [ ]:
# !Pip install pyspark
# !pip install python-dotenv

# Librerias

In [1]:
# Creamos la sessión con spark
from SparkDataLoader import SparkDataLoader
from SparkDataCleaner import SparkDataCleaner
from SparkDataExporter import SparkDataExporter
from pyspark.sql import functions as F
from dotenv import load_dotenv
import os

# Lectura, carga o importación de datos

In [3]:
# Opción 1: Cargar todo automáticamente
loader = SparkDataLoader()
dirty_data=loader.load_all_tables(base_path="./datos", filename_prefix="datos_bi_sucio_bebidas")
loader.show_loaded_tables()
# # Opción 2: Cargar tablas individuales
# loader.load_table('dim_tiempo', './mi_archivo.csv')

2025-11-06 07:44:33,206 - SparkDataLoader - INFO - Inicializando SparkSession: StarSchemaLoader
2025-11-06 07:44:47,771 - SparkDataLoader - INFO - SparkSession inicializada correctamente
2025-11-06 07:44:47,773 - SparkDataLoader - INFO - Spark Version: 4.0.1
2025-11-06 07:44:47,775 - SparkDataLoader - INFO - Iniciando carga de todas las tablas del esquema estrella
2025-11-06 07:44:47,775 - SparkDataLoader - INFO - Ruta base: ./datos
2025-11-06 07:44:47,776 - SparkDataLoader - INFO - Prefijo: datos_bi_sucio_bebidas
2025-11-06 07:44:47,779 - SparkDataLoader - INFO - Cargando dim_tiempo desde: datos\datos_bi_sucio_bebidas_dim_tiempo.csv
2025-11-06 07:44:49,299 - SparkDataLoader - INFO - Tabla dim_tiempo cargada correctamente
2025-11-06 07:44:49,301 - SparkDataLoader - INFO - Cargando dim_tienda desde: datos\datos_bi_sucio_bebidas_dim_tienda.csv
2025-11-06 07:44:49,329 - SparkDataLoader - INFO - Tabla dim_tienda cargada correctamente
2025-11-06 07:44:49,331 - SparkDataLoader - INFO - Carga

In [5]:
print(f"\nTipo de dirty_data: {type(dirty_data)}")
print(f"Es un diccionario con {len(dirty_data)} tablas")
print(f"Claves disponibles: {list(dirty_data.keys())}")


Tipo de dirty_data: <class 'dict'>
Es un diccionario con 4 tablas
Claves disponibles: ['dim_tiempo', 'dim_tienda', 'dim_producto', 'fact_ventas']


In [6]:
dim_tiempo=dirty_data["dim_tiempo"]
dim_tienda=dirty_data["dim_tienda"]
dim_producto=dirty_data["dim_producto"]
facts=dirty_data["fact_ventas"]

# Identificación y limpieza de datos

## Dimensión Tiempo

In [10]:
#Identificación Campos
dim_tiempo.printSchema()
dim_tiempo.show(5)
dim_tiempo.select("Anio", "Mes", "Numero_Semana", "Estacionalidad_Factor").describe().show()
# Ver valores Nullos por columna
dim_tiempo_null = dim_tiempo.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(f"{c}_nulos") for c in dim_tiempo.columns
])
dim_tiempo_null.show()

root
 |-- Fecha_ID: string (nullable = true)
 |-- Fecha: string (nullable = true)
 |-- Anio: integer (nullable = true)
 |-- Mes: integer (nullable = true)
 |-- Nombre_Mes: string (nullable = true)
 |-- Numero_Semana: integer (nullable = true)
 |-- Trimestre: string (nullable = true)
 |-- Estacionalidad_Factor: double (nullable = true)

+--------+----------+----+---+----------+-------------+---------+---------------------+
|Fecha_ID|     Fecha|Anio|Mes|Nombre_Mes|Numero_Semana|Trimestre|Estacionalidad_Factor|
+--------+----------+----+---+----------+-------------+---------+---------------------+
|20230101|2023-01-01|2023|  1|     Enero|           52|   2023Q1|  -0.9981855344718586|
|20230102|2023-01-02|2023|  1|     Enero|            1|   2023Q1|  -0.9970011699250151|
|20230103|2023-01-03|2023|  1|     Enero|            1|   2023Q1|  -0.9955213724144752|
|20230104|2023-01-04|2023|  1|     Enero|            1|   2023Q1|   -0.993746580436178|
|20230105|2023-01-05|2023|  1|     Enero|     

In [12]:
#1.Validacíón e identificación de valores null o erroneos en columna Fecha_ID
print("\n" + "="*80)
print("1. ANÁLISIS DE Fecha_ID (Tipo Mixto)")
print("="*80)

# Detectar tipos de datos
print("\nDetección de tipos en Fecha_ID:")
# Clasificar por patrón
dim_tiempo_tipos = dim_tiempo.withColumn(
    "Fecha_ID_Tipo",
    F.when(F.col("Fecha_ID").rlike("^[0-9]{8}$"), "Numérico_8_digitos")
    .when(F.col("Fecha_ID").rlike("^[0-9]+$"), "Numérico_otro")
    .when(F.col("Fecha_ID").isNull(), "NULL")
    .otherwise("No_numérico")
)

print("\nDistribución de tipos:")
dim_tiempo_tipos.groupBy("Fecha_ID_Tipo").count().orderBy(F.col("count").desc()).show()

# Ejemplos de cada tipo
print("\nEjemplos por tipo:")
for tipo in ["Numérico_8_digitos", "Numérico_otro", "No_numérico", "NULL"]:
    print(f"\n  Tipo: {tipo}")
    ejemplos = dim_tiempo_tipos.filter(F.col("Fecha_ID_Tipo") == tipo).select("Fecha_ID").limit(5)
    if ejemplos.count() > 0:
        ejemplos.show(truncate=False)
    else:
        print("    (Sin ejemplos)")

#---------------------------------------------------------------------------------------------------------------------------------------
#2.Validacíón e identificación de valores null o erroneos en columna Nombre_Mes
print(f"\n{'-'*80}")
print("2. ANÁLISIS DE Nombre_Mes (Formato Diferente Nombre Mes)")
print(f"{'-'*80}")

# Contar elementos distintos
nombre_mes_distinct= dim_tiempo.select("Nombre_Mes").distinct().orderBy("Nombre_Mes").cache()
total_distintos= nombre_mes_distinct.count()
print(f"\nTotal de valores distintos: {total_distintos}")

# Ver con frecuencias 
print("Frecuencia de cada variación:")
dim_tiempo.groupBy("Nombre_Mes").count().orderBy("Nombre_Mes").show(30, truncate=False)


1. ANÁLISIS DE Fecha_ID (Tipo Mixto)

Detección de tipos en Fecha_ID:

Distribución de tipos:
+------------------+-----+
|     Fecha_ID_Tipo|count|
+------------------+-----+
|Numérico_8_digitos|  723|
|       No_numérico|    4|
|              NULL|    3|
+------------------+-----+


Ejemplos por tipo:

  Tipo: Numérico_8_digitos
+--------+
|Fecha_ID|
+--------+
|20230101|
|20230102|
|20230103|
|20230104|
|20230105|
+--------+


  Tipo: Numérico_otro
    (Sin ejemplos)

  Tipo: No_numérico
+----------------+
|Fecha_ID        |
+----------------+
|2024-X-01       |
|2024-X-01       |
|ID_ERRONEO      |
|FECHA_INCORRECTA|
+----------------+


  Tipo: NULL
+--------+
|Fecha_ID|
+--------+
|NULL    |
|NULL    |
|NULL    |
+--------+


--------------------------------------------------------------------------------
2. ANÁLISIS DE Nombre_Mes (Formato Diferente Nombre Mes)
--------------------------------------------------------------------------------

Total de valores distintos: 18
Frecuen

## Dimensión Tienda

In [14]:
#Identificación Campos
dim_tienda.printSchema()
dim_tienda.show(5)
dim_tienda.describe().show()
# Ver valores Nullos por columna
dim_tienda_null = dim_tienda.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(f"{c}_nulos") for c in dim_tienda.columns
])
dim_tienda_null.show()

root
 |-- Tienda_ID: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Formato_Tienda: string (nullable = true)
 |-- Atractivo_Factor: double (nullable = true)

+---------+---------+-------------------+----------------+
|Tienda_ID|   Region|     Formato_Tienda|Atractivo_Factor|
+---------+---------+-------------------+----------------+
|        1|    Norte|       Supermercado|            1.26|
|        2|    Norte|       Supermercado|            1.49|
|        3|   Centro|       Supermercado|             1.4|
|        4|      Sur|       Supermercado|            1.31|
|        5|Occidente|Tienda_Conveniencia|             1.4|
+---------+---------+-------------------+----------------+
only showing top 5 rows
+-------+------------------+------+--------------------+------------------+
|summary|         Tienda_ID|Region|      Formato_Tienda|  Atractivo_Factor|
+-------+------------------+------+--------------------+------------------+
|  count|                50|    50|  

In [16]:
#1.Validacíón e identificación de valores null o erroneos en columna Región
print(f"\n{'-'*80}")
print("1. ANÁLISIS DE Region (Nombres no Homologados)")
print(f"{'-'*80}")

# Contar elementos distintos
nombre_region_distinct= dim_tienda.select("Region").distinct().orderBy("Region").cache()
total_distintos= nombre_region_distinct.count()
print(f"\nTotal de valores distintos: {total_distintos}")

# Ver con frecuencias 
print("Frecuencia de cada variación:")
dim_tienda.groupBy("Region").count().orderBy("Region").show(30, truncate=False)

#2.Validacíón e identificación de valores null o erroneos en columna Formato_Tienda
print(f"\n{'-'*80}")
print("2. ANÁLISIS DE Formato_Tienda (Nombres no Homologados)")
print(f"{'-'*80}")

# Contar elementos distintos 
nombre_region_distinct= dim_tienda.select("Formato_Tienda").distinct().orderBy("Formato_Tienda").cache()
total_distintos= nombre_region_distinct.count()
print(f"\nTotal de valores distintos: {total_distintos}")

# Ver con frecuencias 
print("Frecuencia de cada variación:")
dim_tienda.groupBy("Formato_Tienda").count().orderBy("Formato_Tienda").show(30, truncate=False)   


--------------------------------------------------------------------------------
1. ANÁLISIS DE Region (Nombres no Homologados)
--------------------------------------------------------------------------------

Total de valores distintos: 4
Frecuencia de cada variación:
+---------+-----+
|Region   |count|
+---------+-----+
|Centro   |18   |
|Norte    |14   |
|Occidente|5    |
|Sur      |13   |
+---------+-----+


--------------------------------------------------------------------------------
2. ANÁLISIS DE Formato_Tienda (Nombres no Homologados)
--------------------------------------------------------------------------------

Total de valores distintos: 3
Frecuencia de cada variación:
+--------------------+-----+
|Formato_Tienda      |count|
+--------------------+-----+
|Supermercado        |29   |
|Tienda_Conveniencia |11   |
|Tienda_Especializada|10   |
+--------------------+-----+



## Dimensión Facts

In [18]:
#Identificación Campos
facts.printSchema()
# facts.show(5)
facts.describe("Tienda_ID", "Producto_ID", "Fecha_ID", "Distribucion_Numerica", "Distribucion_Ponderada").show()
facts.describe("Precio", "Out_Of_Stock_Flag", "Ventas_Volumen", "Ventas_Valor").show()
# facts.describe().show()
# Ver valores Nullos por columna
facts_null = facts.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(f"{c}_nulos") for c in facts.columns
])


# facts_null.select("Fecha_ID", "Precio").show()
# facts_null.select("Precio", "Out_Of_Stock_Flag", "Ventas_Volumen", "Ventas_Valor").show()

root
 |-- Tienda_ID: integer (nullable = true)
 |-- Producto_ID: integer (nullable = true)
 |-- Fecha_ID: string (nullable = true)
 |-- Distribucion_Numerica: double (nullable = true)
 |-- Distribucion_Ponderada: double (nullable = true)
 |-- Precio: double (nullable = true)
 |-- Out_Of_Stock_Flag: integer (nullable = true)
 |-- Ventas_Volumen: integer (nullable = true)
 |-- Ventas_Valor: double (nullable = true)

+-------+------------------+------------------+------------------+---------------------+----------------------+
|summary|         Tienda_ID|       Producto_ID|          Fecha_ID|Distribucion_Numerica|Distribucion_Ponderada|
+-------+------------------+------------------+------------------+---------------------+----------------------+
|  count|          40000000|          40000000|          40000000|             40000000|              40000000|
|   mean|        25.5021886|     1050.50450245|2.02356462636302E7|   0.7499863614999879|    0.7999894627439181|
| stddev|14.4305357286

In [20]:
#1.Validacíón e identificación de valores null o erroneos en columna Fecha_ID
print(f"\n{'-'*80}")
print("1. ANÁLISIS DE Fecha_ID")
print(f"{'-'*80}")

# Clasificar por patrón
facts_fecha_id_tipos = facts.withColumn(
    "Fecha_ID_Tipo",
    F.when(F.col("Fecha_ID").rlike("^[0-9]{8}$"), "Numérico_8_digitos")
    .when(F.col("Fecha_ID").rlike("^[0-9]+$"), "Numérico_otro")
    .when(F.col("Fecha_ID").isNull(), "NULL")
    .otherwise("No_numérico")
)

print("\nDistribución de tipos:")
facts_fecha_id_tipos.groupBy("Fecha_ID_Tipo").count().orderBy(F.col("count").desc()).show()


--------------------------------------------------------------------------------
1. ANÁLISIS DE Fecha_ID
--------------------------------------------------------------------------------

Distribución de tipos:
+------------------+--------+
|     Fecha_ID_Tipo|   count|
+------------------+--------+
|Numérico_8_digitos|40000000|
+------------------+--------+



In [22]:
#2.Validacíón e identificación de valores variables numéricas
#-------------------------------Venta_Normal
print(f"\n{'-'*80}")
print("1. Venta_Normal")
print(f"{'-'*80}")

total_registros = facts.count()
vta_normal=facts.filter(
    (F.col("Ventas_Valor")>0) & 
    (F.col("Ventas_Volumen")>0) & 
    (F.col("Precio")>0) &
    (F.col("Out_Of_Stock_Flag") ==0)
          )
porcentaje_vts_normal=vta_normal.count()/ total_registros

print(f" % Vtas Normales : {porcentaje_vts_normal:.2%}")
vta_normal.show(5)

#-------------------------------Devolución
print(f"\n{'-'*80}")
print("2. Devolución")
print(f"{'-'*80}")

devolucion=facts.filter(
    (F.col("Ventas_Volumen")<0) &
    (F.col("Precio")>0)
          )
porcentaje_devolucion=devolucion.count()/ total_registros

print(f" % Devolución : {porcentaje_devolucion:.2%} (vts Vol < 0 & Precio > 0)")
devolucion.show(5)

#-------------------------------Ajuste de Inventario
print(f"\n{'-'*80}")
print("3. Ajuste de Inventario")
print(f"{'-'*80}")

ajuste_de_inventario=facts.filter(
    (F.col("Ventas_Volumen")<0) &
    (F.col("Precio")>0) &
    (F.col("Out_Of_Stock_Flag") ==1)
          )
porcentaje_ajuste_de_inventario=ajuste_de_inventario.count()/ total_registros

print(f" % Ajuste de Inventario : {porcentaje_ajuste_de_inventario:.2%} (vts Vol <0 & Precio > 0 OOS =1)")
ajuste_de_inventario.show(5)

#-------------------------------Venta perdida
print(f"\n{'-'*80}")
print("4. Venta Perdida")
print(f"{'-'*80}")

venta_perdida=facts.filter(
    (F.col("Ventas_Volumen")==0) &
    (F.col("Out_Of_Stock_Flag") ==1) &
    (F.col("Ventas_Valor")==0) | (F.col("Ventas_Valor").isNull())
          )
porcentaje_venta_perdida=venta_perdida.count()/ total_registros

print(f" % Venta Perdida : {porcentaje_venta_perdida:.2%} (vts Vol=0 & OOS=1 & vts Val =0 | vts Val = null)")
venta_perdida.show(5)

#-------------------------------Registro sin Venta
print(f"\n{'-'*80}")
print("5. Registro sin venta")
print(f"{'-'*80}")

registro_sin_venta=facts.filter(
    (F.col("Ventas_Volumen")==0) &
    (F.col("Out_Of_Stock_Flag") ==0) &
    (F.col("Ventas_Valor")==0) | (F.col("Ventas_Valor").isNull())
          )
porcentaje_registro_sin_venta=registro_sin_venta.count()/ total_registros

print(f" % Registro sin Venta : {porcentaje_registro_sin_venta:.2%} (vts Vol=0 & OOS=0 & vts Val =0 | vts Val = null)")
venta_perdida.show(5)

Total_errores= porcentaje_vts_normal + porcentaje_devolucion + porcentaje_ajuste_de_inventario + porcentaje_venta_perdida +  porcentaje_registro_sin_venta



print(f"Total Escencarios {Total_errores:2%}")


--------------------------------------------------------------------------------
1. Venta_Normal
--------------------------------------------------------------------------------
 % Vtas Normales : 45.87%
+---------+-----------+--------+---------------------+----------------------+------+-----------------+--------------+------------+
|Tienda_ID|Producto_ID|Fecha_ID|Distribucion_Numerica|Distribucion_Ponderada|Precio|Out_Of_Stock_Flag|Ventas_Volumen|Ventas_Valor|
+---------+-----------+--------+---------------------+----------------------+------+-----------------+--------------+------------+
|       30|       1055|20240822|                 0.62|                  0.73|  1.04|                0|            25|        26.0|
|       47|       1089|20230524|                  0.6|                  0.63|  4.67|                0|            99|      462.33|
|       20|       1014|20230809|                 0.76|                  0.68|  1.04|                0|            58|       60.32|
|       2

# Proceso de Limpieza de datos

## Escenario 1

In [ ]:
#Se inicia la función para limpiar el conjunto de datos
cleaner=SparkDataCleaner(dataframes=dirty_data)
dim_tiempo= cleaner.clean_dim_tiempo()
dim_tienda=cleaner.clean_dim_tienda()
facts = cleaner.clean_fact_ventas(
    remove_null_precio=False,
    remove_null_volumen=True,
    impute_distribucion=True,
    recalculate_valor=True,
    remove_outliers=False
)

# Pipeline (Exportación de datos)

In [7]:
from SparkDataLoader import SparkDataLoader
from SparkDataCleaner import SparkDataCleaner
from SparkDataExporter import SparkDataExporter
load_dotenv()

# ============================================================================
# CONFIGURACIÓN (AJUSTA ESTOS VALORES)
# ============================================================================

# Ruta al JAR (para cargar en Spark)
JDBC_DRIVER_PATH = r"C:\sqljdbc_13.2.1.0_enu\sqljdbc_13.2\enu\jars\mssql-jdbc-13.2.1.jre8.jar"

# Conexión SQL Server
JDBC_URL = r"jdbc:sqlserver://ALTALU87:1433;databaseName=DB_Beverages;encrypt=true;trustServerCertificate=true"
JDBC_USER = os.getenv("SQL_DB_USER")
JDBC_PASSWORD =os.getenv("SQL_DB_PASS")

# Nombre de la clase del driver (NO CAMBIAR ESTO)
JDBC_DRIVER_CLASS = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

# ============================================================================
# PIPELINE COMPLETO
# ============================================================================

print("="*80)
print("PIPELINE: CARGA → LIMPIEZA → EXPORTACIÓN")
print("="*80)

# ---------------------------------------------------------------------------
# PASO 1: Cargar datos CON el JAR
# ---------------------------------------------------------------------------
print("\n[1/4] Cargando datos...")

loader = SparkDataLoader(
    app_name="PipelineCompleto",
    driver_memory="8g"
)

# IMPORTANTE: Agregar el JAR manualmente a la sesión de Spark
# loader.spark.sparkContext.addPyFile(JDBC_DRIVER_PATH)

dirty_data = loader.load_all_tables(
    base_path="./datos",
    filename_prefix="datos_bi_sucio_bebidas"
)

loader.show_loaded_tables()

# ---------------------------------------------------------------------------
# PASO 2: Limpiar datos
# ---------------------------------------------------------------------------
print("\n[2/4] Limpiando datos...")

cleaner = SparkDataCleaner(dataframes=dirty_data)

clean_data = cleaner.clean_all(
    enforce_integrity=True,
    fact_options={
        'remove_null_precio': False,
        'remove_null_volumen': True,
        'impute_distribucion': True,
        'recalculate_valor': True,
        'remove_outliers': False
    }
)

cleaner.print_cleaning_summary()

# ---------------------------------------------------------------------------
# PASO 3: Exportar a SQL Server
# ---------------------------------------------------------------------------
print("\n[3/4] Exportando a SQL Server...")

exporter = SparkDataExporter(
    jdbc_url=JDBC_URL,
    jdbc_user=JDBC_USER,
    jdbc_password=JDBC_PASSWORD,
    jdbc_driver=JDBC_DRIVER_CLASS  # ← NOMBRE DE CLASE (no la ruta)
)

# Exportar todas las tablas
results_sql = exporter.export_multiple_tables_to_sql(
    tables=clean_data,
    table_prefix="clean_",
    mode="overwrite",
    batchsize=10000,
    max_partitions=4,  # Ajusta según tu servidor
    retries=3
)

---------------------------------------------------------------------------
PASO 4: Exportar a CSV (backup)
---------------------------------------------------------------------------
print("\n[4/4] Exportando a CSV...")

results_csv = exporter.export_multiple_tables_to_csv(
    tables=clean_data,
    base_path="./export_csv",
    num_files=1,
    compression=None
)

# ---------------------------------------------------------------------------
# RESUMEN FINAL
# ---------------------------------------------------------------------------
print("\n" + "="*80)
print("RESUMEN DE EXPORTACIONES")
print("="*80)

exporter.print_export_summary()

print("\n✓ Pipeline completado exitosamente")

# Verificar resultados
all_sql_success = all(results_sql.values())
all_csv_success = all(results_csv.values())

if all_sql_success and all_csv_success:
    print("✓✓✓ TODAS las exportaciones fueron exitosas")
else:
    if not all_sql_success:
        failed_sql = [k for k, v in results_sql.items() if not v]
        print(f"⚠️ SQL fallidas: {failed_sql}")
    if not all_csv_success:
        failed_csv = [k for k, v in results_csv.items() if not v]
        print(f"⚠️ CSV fallidas: {failed_csv}")

# Detener Spark
loader.stop()

2025-11-03 21:13:02,773 - SparkDataLoader - INFO - Inicializando SparkSession: PipelineCompleto
2025-11-03 21:13:02,950 - SparkDataLoader - INFO - SparkSession inicializada correctamente
2025-11-03 21:13:02,951 - SparkDataLoader - INFO - Spark Version: 4.0.1
2025-11-03 21:13:02,952 - SparkDataLoader - INFO - Iniciando carga de todas las tablas del esquema estrella
2025-11-03 21:13:02,953 - SparkDataLoader - INFO - Ruta base: ./datos
2025-11-03 21:13:02,953 - SparkDataLoader - INFO - Prefijo: datos_bi_sucio_bebidas
2025-11-03 21:13:02,957 - SparkDataLoader - INFO - Cargando dim_tiempo desde: datos\datos_bi_sucio_bebidas_dim_tiempo.csv


PIPELINE: CARGA → LIMPIEZA → EXPORTACIÓN

[1/4] Cargando datos...


2025-11-03 21:13:03,141 - SparkDataLoader - INFO - Tabla dim_tiempo cargada correctamente
2025-11-03 21:13:03,144 - SparkDataLoader - INFO - Cargando dim_tienda desde: datos\datos_bi_sucio_bebidas_dim_tienda.csv
2025-11-03 21:13:03,160 - SparkDataLoader - INFO - Tabla dim_tienda cargada correctamente
2025-11-03 21:13:03,164 - SparkDataLoader - INFO - Cargando dim_producto desde: datos\datos_bi_sucio_bebidas_dim_producto.csv
2025-11-03 21:13:03,182 - SparkDataLoader - INFO - Tabla dim_producto cargada correctamente
2025-11-03 21:13:03,185 - SparkDataLoader - INFO - Cargando fact_ventas desde: datos\datos_bi_sucio_bebidas_fact_ventas.csv
2025-11-03 21:13:03,204 - SparkDataLoader - INFO - Tabla fact_ventas cargada correctamente
2025-11-03 21:13:03,204 - SparkDataLoader - INFO - Carga completada: 4/4 tablas
2025-11-03 21:13:03,205 - SparkDataLoader - INFO - 
2025-11-03 21:13:03,206 - SparkDataLoader - INFO - RESUMEN DE TABLAS CARGADAS
2025-11-03 21:13:03,206 - SparkDataLoader - INFO - ====


[2/4] Limpiando datos...


2025-11-03 21:13:06,794 - SparkDataCleaner - INFO - Fechas convertidas exitosamente: 730 (100.00%)
2025-11-03 21:13:06,903 - SparkDataCleaner - INFO - Fecha_IDs inválidos encontrados: 4
2025-11-03 21:13:06,904 - SparkDataCleaner - WARNING - Mostrando ejemplos de Fecha_IDs inválidos:
2025-11-03 21:13:06,999 - SparkDataCleaner - INFO - Intentando corregir Fecha_IDs inválidos usando columna Fecha...
2025-11-03 21:13:07,172 - SparkDataCleaner - INFO - Fecha_IDs corregidos desde columna Fecha: 4


...

2025-11-03 21:16:46,678 - SparkDataCleaner - INFO - 
2025-11-03 21:16:46,679 - SparkDataCleaner - INFO - APLICANDO INTEGRIDAD REFERENCIAL
2025-11-03 21:16:46,681 - SparkDataCleaner - INFO - ============================================================
2025-11-03 21:17:04,347 - SparkDataCleaner - INFO - Registros con Fecha_ID huérfano eliminados: 0
2025-11-03 21:17:14,928 - SparkDataCleaner - INFO - Registros con Tienda_ID huérfano eliminados: 0
2025-11-03 21:17:25,546 - SparkDataCleaner - INFO - Registros con Producto_ID huérfano eliminados: 0
2025-11-03 21:17:25,547 - SparkDataCleaner - INFO - 
Total registros eliminados: 0 (0.00%)
2025-11-03 21:17:25,549 - SparkDataCleaner - INFO - Registros finales: 40,000,000
2025-11-03 21:17:25,549 - SparkDataCleaner - INFO - 
2025-11-03 21:17:25,550 - SparkDataCleaner - INFO -                     LIMPIEZA COMPLETADA
2025-11-03 21:17:25,552 - SparkDataCleaner - INFO - ======================================================================
2025-11-03


                    RESUMEN DE LIMPIEZA

DIM_TIEMPO
------------------------------------------------------------
  Registros iniciales:           730
  Registros finales:             730
  Registros eliminados:            0
  Porcentaje eliminado:         0.00%

DIM_TIENDA
------------------------------------------------------------
  Registros iniciales:            50
  Registros finales:              50
  Registros eliminados:            0
  Porcentaje eliminado:         0.00%

DIM_PRODUCTO
------------------------------------------------------------
  Registros iniciales:           100
  Registros finales:             100
  Registros eliminados:            0
  Porcentaje eliminado:         0.00%

FACT_VENTAS
------------------------------------------------------------
  Registros iniciales:    40,000,000
  Registros finales:      40,000,000
  Registros eliminados:            0
  Porcentaje eliminado:         0.00%

  Integridad Referencial:
    - Fecha_ID huérfanos:             0
 

2025-11-03 21:17:26,307 - SparkDataExporter - INFO - ✅✅✅ EXPORTACIÓN EXITOSA A SQL SERVER ✅✅✅
2025-11-03 21:17:26,308 - SparkDataExporter - INFO - 📊 Total exportado: 730 registros
2025-11-03 21:17:26,309 - SparkDataExporter - INFO - 📍 Tabla destino: clean_dim_tiempo
2025-11-03 21:17:26,310 - SparkDataExporter - INFO - ⚙️ Modo: overwrite
2025-11-03 21:17:26,311 - SparkDataExporter - INFO - ⏱️ Tiempo total en export_to_sql: 0.75 segundos (0.01 minutos)
2025-11-03 21:17:26,312 - SparkDataExporter - INFO - 
--- Exportando: clean_dim_tienda ---
2025-11-03 21:17:26,312 - SparkDataExporter - INFO - ⏳ Iniciando: export_to_sql
2025-11-03 21:17:26,374 - SparkDataExporter - INFO - 📊 Total de registros a exportar: 50
2025-11-03 21:17:26,375 - SparkDataExporter - INFO - 🔧 Reparticionando en 4 particiones...
2025-11-03 21:17:26,378 - SparkDataExporter - INFO - 🚀 Intento 1/3: Exportando a SQL Server → clean_dim_tienda
2025-11-03 21:17:26,622 - SparkDataExporter - INFO - ✅✅✅ EXPORTACIÓN EXITOSA A SQL 


RESUMEN DE EXPORTACIONES

                    RESUMEN DE EXPORTACIONES

clean_dim_tiempo:
  Estado: ✓ Exitoso
  Registros: 730
  Modo: overwrite
  Intentos: 1

clean_dim_tienda:
  Estado: ✓ Exitoso
  Registros: 50
  Modo: overwrite
  Intentos: 1

clean_dim_producto:
  Estado: ✓ Exitoso
  Registros: 100
  Modo: overwrite
  Intentos: 1

clean_fact_ventas:
  Estado: ✓ Exitoso
  Registros: 40,000,000
  Modo: overwrite
  Intentos: 1


✓ Pipeline completado exitosamente


NameError: name 'results_csv' is not defined